In [0]:
%sql
-- 00_register_tables.sql (Unity Catalog version)

-- Switch to Unity Catalog's main catalog
USE CATALOG main;

-- Create schema (Unity Catalog will auto-manage storage location)
CREATE SCHEMA IF NOT EXISTS demo
COMMENT 'Demo schema for geospatial data';

USE SCHEMA demo;

-- raw points
CREATE TABLE IF NOT EXISTS points_raw
USING DELTA
LOCATION 'abfss://raw@trimblegeospatialdemo.dfs.core.windows.net/points';

-- Processed points (tiled, point-level) - External table
CREATE TABLE IF NOT EXISTS demo_processed_points_tiled
USING DELTA
LOCATION 'abfss://processed@trimblegeospatialdemo.dfs.core.windows.net/points_tiled';

-- Aggregated tile stats (tile-level summary + bbox) - External table
CREATE TABLE IF NOT EXISTS demo_agg_tile_stats
USING DELTA
LOCATION 'abfss://aggregated@trimblegeospatialdemo.dfs.core.windows.net/tile_stats';

-- Aggregated tile water mask (water tiles only) - External table
CREATE TABLE IF NOT EXISTS demo_agg_tile_water_mask
USING DELTA
LOCATION 'abfss://aggregated@trimblegeospatialdemo.dfs.core.windows.net/tile_water_mask';

In [0]:
%sql
-- pipeline_runs for job status logging
CREATE TABLE IF NOT EXISTS main.demo.pipeline_runs (
  siteId            STRING,
  runId             STRING,
  pipelineName      STRING,
  status            STRING,        -- QUEUED | RUNNING | SUCCEEDED | FAILED | CANCELLED
  startedAt         TIMESTAMP,
  finishedAt        TIMESTAMP,
  errorMessage      STRING,
  triggeredBy       STRING,        -- user / service / scheduler
  createdAt         TIMESTAMP
)
USING DELTA
PARTITIONED BY (siteId);

In [0]:
%sql
-- 03_01_register_tables.sql (Unity Catalog version)
USE CATALOG main;
USE SCHEMA demo;

-- Surface cells aggregation table
CREATE TABLE IF NOT EXISTS demo_agg_surface_cells
USING DELTA
LOCATION 'abfss://aggregated@trimblegeospatialdemo.dfs.core.windows.net/surfaces/surface_cells';

-- Surface patches aggregation table
CREATE TABLE IF NOT EXISTS demo_agg_surface_patches
USING DELTA
LOCATION 'abfss://aggregated@trimblegeospatialdemo.dfs.core.windows.net/surfaces/surface_patches';

In [0]:
# Check current state of Unity Catalog demo schema and tables
print("=== 1. Check if demo schema exists in Unity Catalog ===")
try:
    # Check schemas in main catalog
    schemas = spark.sql("SHOW SCHEMAS IN main").collect()
    demo_exists = any(s.databaseName == 'demo' for s in schemas)
    if demo_exists:
        print("✅ main.demo schema exists")
    else:
        print("❌ main.demo schema does NOT exist")
        print("\nAvailable schemas in main catalog:")
        for s in schemas:
            print(f"  - main.{s.databaseName}")
except Exception as e:
    print(f"❌ Error: {e}")

print("\n=== 2. Check tables in main.demo schema ===")
try:
    tables = spark.sql("SHOW TABLES IN main.demo").collect()
    if tables:
        print(f"Found {len(tables)} tables in main.demo schema:")
        for t in tables:
            # Use 'database' and 'tableName' fields (DBR 16.4 compatible)
            print(f"  ✅ main.{t.database}.{t.tableName}")
    else:
        print("❌ No tables found in main.demo schema")
except Exception as e:
    print(f"❌ Error: {e}")

print("\n=== 3. Check if data files exist in ADLS ===")
paths_to_check = [
    ("processed_points_tiled", "abfss://processed@trimblegeospatialdemo.dfs.core.windows.net/points_tiled"),
    ("agg_tile_stats", "abfss://aggregated@trimblegeospatialdemo.dfs.core.windows.net/tile_stats"),
    ("agg_tile_water_mask", "abfss://aggregated@trimblegeospatialdemo.dfs.core.windows.net/tile_water_mask"),
    ("agg_surface_cells", "abfss://aggregated@trimblegeospatialdemo.dfs.core.windows.net/surfaces/surface_cells"),
    ("agg_surface_patches", "abfss://aggregated@trimblegeospatialdemo.dfs.core.windows.net/surfaces/surface_patches"),
]

for name, path in paths_to_check:
    try:
        df = spark.read.format("delta").load(path)
        count = df.count()
        print(f"  ✅ {name}: {count:,} rows exist in ADLS")
    except Exception as e:
        if "PATH_NOT_FOUND" in str(e):
            print(f"  ❌ {name}: Data files do NOT exist at {path}")
        else:
            print(f"  ⚠️ {name}: Error - {str(e)[:100]}")

print("\n=== 4. Summary ===")
print("Tables registered in Unity Catalog (main.demo) are persistent.")
print("They will NOT be lost after cluster restart.")
print("Use Catalog Explorer (left sidebar) or SHOW TABLES to view registered tables.")